<a href="https://colab.research.google.com/github/jonghhhh/lecture_colabs/blob/main/%ED%85%8D%EC%8A%A4%ED%8A%B8_%EC%9C%A0%EC%82%AC%EB%8F%84_%EC%A0%9C%EB%AA%A9%EB%8C%80%ED%91%9C%EC%84%B1_%ED%91%9C%EC%A0%88_%EB%B6%84%EC%84%9D_051724.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# KPF-BERT 활용 문서 분석

https://github.com/KPF-bigkinds/KPF-BERT

## tokenize: 문장을 토큰(최대 512개)으로 분리

In [ ]:
from transformers import AutoTokenizer

# 토크나이저 로드
tokenizer = AutoTokenizer.from_pretrained("jinmang2/kpfbert")

# 토크나이즈할 문장
sentence = "이것은 토크나이저 테스트 문장입니다."

# 문장 토크나이징
tokens = tokenizer.tokenize(sentence)

# 토크나이즈된 토큰을 토큰 ID로 변환
token_ids = tokenizer.convert_tokens_to_ids(tokens)

# 결과 출력
print(f"토크나이즈된 토큰: {tokens}")
print(f"토크나이즈된 토큰의 ID: {token_ids}")

## encoding: 문장의 특성을 담은 cls의 벡터 값(768차원) 추출

In [ ]:
from transformers import AutoTokenizer, AutoModel
import torch

# 모델과 토크나이저 로드
tokenizer = AutoTokenizer.from_pretrained("jinmang2/kpfbert")   # kpfberteh max token = 512
model = AutoModel.from_pretrained("jinmang2/kpfbert")

In [ ]:
# 함수 encode: 문장을 토크나이즈하고 모델을 통해 벡터화

def encode(text):
    inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True, max_length=512)
    with torch.no_grad():
        outputs = model(**inputs)
    # 첫 번째 토큰 ([CLS] 토큰)의 벡터를 1차원으로 변환
    return outputs.last_hidden_state[:, 0, :].squeeze().numpy()

# 실습
vec = encode('나는 파이썬을 좋아한다')
len(vec)

## 문장간 유사도: 코사인유사도 이용

In [ ]:
from transformers import AutoTokenizer, AutoModel
import torch
from scipy.spatial.distance import cosine

# 모델과 토크나이저 로드
tokenizer = AutoTokenizer.from_pretrained("jinmang2/kpfbert")   # kpfberteh max token = 512
model = AutoModel.from_pretrained("jinmang2/kpfbert")

In [ ]:
# 문장 2개(2개 벡터) 간 코사인 유사도 측정

text1 = "이 문장은 샘플 문장입니다."
text2 = "이것은 비교할 다른 샘플 문장입니다."

# 위 encode 함수 사용해 각 문장을 토크나이즈하고 모델을 통해 벡터화
vec1 = encode(text1)
vec2 = encode(text2)

# 코사인 유사도 계산
similarity = 1 - cosine(vec1, vec2)
print(f"문장 유사도: {similarity}")

In [ ]:
# 복수의 문장들(복수의 벡터들) 간 코사인 유사도를 행렬로 구하는 함수: cosine_similarity_matrix

import numpy as np
from scipy.spatial.distance import cosine

def cosine_similarity_matrix(vectors):
    n = len(vectors)
    similarity_matrix = np.zeros((n, n))
    for i in range(n):
        for j in range(i, n):
            similarity = 1 - cosine(vectors[i], vectors[j])
            similarity_matrix[i, j] = similarity
            similarity_matrix[j, i] = similarity  # 대칭성을 이용
    return similarity_matrix

# 실습
v1 = np.array([1, 2, 3])
v2 = np.array([4, 5, 6])
v3 = np.array([7, 8, 9])
v4 = np.array([1, 0, 0])
vectors = [v1, v2, v3, v4]
similarity_matrix = cosine_similarity_matrix(vectors)
print("코사인 유사도 행렬:")
print(similarity_matrix)

## 문장 분리: kiwipiepy 활용

In [ ]:
# 문장 분리 함수: sent_split

! pip install kiwipiepy

from kiwipiepy import Kiwi
kiwi = Kiwi()

def sent_split(txts):
    sents=kiwi.split_into_sents(txts)
    sents=[s[0] for s in sents]
    for i,s in enumerate(sents):
        for t in ['"고','"라고','"며', '"라며', '"면서','"라면서']:  #연결조사로 이어진 부분이 분리됨. 이를 연결시키는 장치.
            if s.startswith(t):
                sents[i-1]=sents[i-1]+s
                del sents[i]
            else:
                pass
    return sents

# 실습
txt='철수는 어려운 파이썬을 열심히 학습합니다 영희도 파이썬을 좋아할까요?'
sent_split(txt)


# KPF-SBERT 활용

## 문서(기사) 내 일관성: 제목의 대표성과 문장들의 수렴성
- https://github.com/KPFBERT/kpfSBERT
- https://huggingface.co/bongsoo/kpf-sbert-128d-v1 (huggingface에서 로드 가능)
- KPF-SBERT의 벡터를 768차원에서 128차원으로 줄여 유사도 값을 대체적으로 낮춤. 비교 분석을 용이하게 해줌.

In [ ]:
! pip install sentence-transformers

In [ ]:
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
import pandas as pd
import numpy as np

# 모델 로드
s_model = SentenceTransformer('bongsoo/kpf-sbert-128d-v1')

def coherence_within_docs(article, sent_max=10):   # 문서에서 앞에서(제목이 첫번째 문장) 몇개 문장까지 사용할 것인지 설정
    if sent_max:
        txts=sent_split(article)[:sent_max]
    else:
        txts=sent_split(article)
    encoded_txts = s_model.encode(txts)
    sim_matrix=cosine_similarity_matrix(encoded_txts)
    sim_matrix[sim_matrix == 1] = np.nan
    matrix_df=round(pd.DataFrame(sim_matrix), 4)
    title_represent=round(matrix_df.iloc[0].mean(), 4)   # 기사의 첫번째 문장이 title이라고 할 때, title이 다른 문장들과 맺는 유사도 평균 -> 제목의 내용 요약 수준(대표성)
    total_coherence=round(matrix_df.mean().mean(), 4)    # 제목을 포함해 모든 문장들 간의 유사도 평균 -> 기사의 의미상 일관성(수렴성) 수준
    return (title_represent, total_coherence, matrix_df)

In [ ]:
article1 = '''풍부한 첫 느낌, 깔끔한 여운…프리미엄 커피 라이프 ‘카누 원두커피’.
최근 커피를 즐기는 인구가 늘어나면서 프리미엄 원두커피에 대한 수요가 증가하고 있다. 지난해 대한민국 대표 커피전문기업 동서식품이 선보인 ‘카누(KANU) 원두커피’는 원두 본연의 맛과 향을 즐길 수 있는 제품으로 총 6종(홀빈 3종·분쇄 원두 3종)을 갖춰 취향대로 골라 마실 수 있다.
카누 원두커피 6종은 ‘FIRST BOLD, LAST CLEAN(풍부한 첫 느낌, 깔끔한 여운)’이라는 콘셉트로 입안 가득 퍼지는 풍부하고 진한 풍미와 목 넘김 후에 느껴지는 깔끔한 맛을 구현한 것이 특징이다.
중남미와 아프리카 지역의 프리미엄 원두를 사용하고 원두마다 최적의 로스팅 프로파일을 적용해 고유의 풍미와 개성을 극대화했다.
▶카누 실키 베이지 ▶카누 크리미 버건디 ▶카누 벨베티 블랙 등 3가지 플레이버를 각각 분쇄 원두(Ground Coffee)와 홀빈(Whole Bean: 분쇄하지 않은 원두) 2가지 타입으로 선보여 소비자 선택의 폭을 넓혔다. 제품 타입에 따라 에스프레소 방식이나 핸드드립, 콜드브루 등 다양한 방식으로 즐길 수 있다.
‘카누 실키 베이지’는 원두를 라이트 로스팅해 과일류에서 느껴지는 매력적인 산미가 특징이다. ‘카누 크리미 버건디’는 미디엄 로스트 원두로 견과류의 진하고 고소한 풍미와 적당한 산미를 느낄 수 있고, ‘카누 벨베티 블랙’은 다크 로스트 원두의 묵직한 무게감 속에 스모키한 향과 달콤하고 쌉싸름한 풍미가 돋보인다.
또한 동서식품은 최근 대용량 신제품 ‘카누 원두커피 카페 블렌드 다크로스팅’(1.13kg)을 론칭하고 전국 코스트코 매장에 입점했다. 카누 카페 블렌드 다크로스팅은 홀빈 타입으로 커피전문점을 운영하는 사업자나 대용량 원두를 원하는 소비자들을 위해 출시한 제품이다.
동서식품은 지난 6일과 7일 이틀간 서울 용산구 한남동의 맥심 브랜드 체험공간 ‘맥심플랜트’에서 월드 바리스타 챔피언 엄보람 바리스타와 함께 ‘카누 원두커피 컬래버레이션 이벤트’를 진행했다.
이번 이벤트는 카누 프리미엄 원두커피 ‘카페 블렌드’ 론칭과 함께 카누 원두커피의 우수한 품질을 알리기 위해 기획됐다. 행사에 참여한 엄보람 바리스타는 지난 2023년 월드 바리스타 챔피언십(WBC)에서 우승한 세계적인 바리스타다.
‘카누 원두커피 컬래버레이션 이벤트’에서는 엄보람 바리스타가 직접 ‘카누 카페 블렌드 다크 로스팅’과 ‘카누 실키 베이지’ 원두를 활용한 핸드드립과 콜드브루 레시피를 소개하는 원데이 클래스와 함께 방문객 대상 샘플링과 시음행사가 진행됐다.
동서식품 김민수 마케팅 매니저는 “카누 원두커피는 프리미엄 원두커피로 산지별 최고 수준의 원두만을 사용했으며 풍부한 첫맛과 깔끔한 여운을 갖춰 원두커피 입문자부터 애호가까지 모두 만족하게 할 수 있는 제품”이라며 “카누 원두커피와 함께 프리미엄 커피 라이프를 즐겨보길 바란다”고 말했다.'''
# 제1조②항 사회·경제 세력으로부터의 독립과 제3조⑥항 보도자료 검증으로 주의 조치

In [ ]:
article2 = '''동서식품, ‘카누 원두커피’ 6종 출시.
풍부한 첫 느낌과 깔끔한 여운… 프리미엄 원두커피의 정수.
동서식품, ‘카누 원두커피’ 6종 출시.
다양한 풍미의 프리미엄 원두커피를 원하는 소비자 겨냥, 3가지 플레이버를 각 2가지 타입으로 출시.
각 원두마다 최적의 로스팅 프로파일을 적용해 원두 고유의 풍미와 개성 극대화.
대한민국 대표 커피전문기업 동서식품(대표 김광수)은 프리미엄 원두 본연의 맛과 향을 즐길 수 있는 ‘카누 원두커피’ 6종을 출시했다고 밝혔다.
이번 신제품은 카누(KANU) 브랜드의 첫 원두커피로 커피 애호가와 홈카페족은 물론 언제 어디서나 풍부한 맛의 프리미엄 원두커피를 즐기고 싶어하는 소비자들이 기대하는 맛과 속성을 반영한 제품이다.
카누 원두커피 6종은 ‘FIRST BOLD, LAST CLEAN(풍부한 첫 느낌, 깔끔한 여운)’이라는 콘셉트로 입안 가득 퍼지는 풍부하고 진한 풍미와 목 넘김 후에 느껴지는 깔끔한 맛을 구현한 것이 특징이다. 중남미와 아프리카 지역의 프리미엄 원두를 사용하고 각 원두마다 최적의 로스팅 프로파일을 적용해 고유의 풍미와 개성을 극대화했다.
신제품은 ▲카누 실키 베이지 ▲카누 크리미 버건디 ▲카누 벨베티 블랙 등 3가지 플레이버를 각각 분쇄 원두(Ground Coffee)와 홀빈(Whole Bean: 분쇄하지 않은 원두) 2가지 타입으로 선보여 소비자 선택의 폭을 넓혔다.
‘카누 실키 베이지’는 원두를 라이트 로스팅해 과일류에서 느껴지는 매력적인 산미가 특징이다. ‘카누 크리미 버건디’는 미디엄 로스트 원두로 견과류의 진하고 고소한 풍미와 적당한 산미를 느낄 수 있고, ‘카누 벨베티 블랙’은 다크 로스트 원두의 묵직한 무게감 속에 스모키한 향과 달콤쌉싸름한 풍미를 경험할 수 있다.
제품 타입에 따라 에스프레소 방식이나 핸드드립, 콜드브루 등 다양한 방식으로 즐길 수 있다. 카누 원두커피 6종의 가격은 1팩(200g)당 전국 대형마트 기준 12,990원이다.
동서식품 김민수 마케팅 매니저는 “이번 신제품은 각 원두의 특징이 명확하면서도 풍부한 첫 맛과 깔끔한 여운을 갖춰 원두커피 입문자부터 애호가까지 모두 만족시킬 수 있는 제품”이라며 "카누 원두커피와 함께 집, 사무실 등 언제 어디서나 여유로운 커피 라이프를 즐기시기 바란다”고 말했다.'''
 # 동서식품 보도자료

In [ ]:
article3 = ''''원두값 급등에 커피가격 올려야…동네카페 걱정'.
국제 원두 가격이 급등한 가운데 국내 카페 자영업자들의 시름도 깊어지고 있다. 사진은 광주 한 카페의 메뉴판.
“저같이 개인이 운영하는 카페는 저렴하게 판매해야만 손님이 오는데 최근 들어 거래처에서 원두 소매값을 올린다는 말이 있어 불안합니다.”
광주 동구 학원가에서 소규모 개인 카페를 운영 중인 김모(52)씨는 요즘 아이스아메리카노 가격을 올릴지 고민 중이다. 최근 급격히 오른 원두 가격에 도매상들이 하나둘 가격을 올리기 시작하면서 가격 조정이 필요하기 때문이다.
김씨는 “아직 거래처에서 원두 가격을 올리지 않았지만, 곧 올린다는 소문이 돈다. 개인 카페이다 보니 저렴한 가격에 커피를 판매하고 있는데 우윳값도 이미 많이 올라 여기에 원두값까지 오르면 매우 힘들어진다”며 “주변에 워낙 카페가 많다 보니 가격을 올리게 되면 가격 경쟁력이 떨어져 고민된다. 마지막까지 심사숙고해 가격을 올릴지 결정할 예정이다”고 한숨을 내쉬었다.
시원한 커피 수요가 늘어나는 여름철이 다가오지만 카페 자영업자들은 비상이다. 이상기후로 커피 생산량이 급감하면서 원두값이 고공행진을 기록하며 도·소매상들이 원두 가격을 올린 까닭이다.
7일 한국농수산식품유통공사(aT) 식품산업통계정보에 따르면 런던국제금융선물거래소(LIFFE) 기준 로부스타 원두 1톤 가격은 올해 평균 3427.19달러로 지난해(2492.82달러) 대비 37.48% 급증했다. 기존 1000달러~2000달러 사이를 유지하던 로부스타 가격은 올해 1월 3236.5달러로 치솟으며 전년동월(1962.86달러) 대비 무려 64.89% 뛰었다. 원두값은 꾸준히 증가 곡선을 그리며 지난 4월에는 3938.86달러 기록하며 16년 만에 최고가를 경신했다.
원두가격 상승은 세계 로부스타 공급량의 3분의1을 차지하는 베트남에 가뭄이 지속되면서 원두 생산량이 급감한 것이 주된 요인이다. 로부스타 원두는 저렴한 가격으로 인스턴트 커피, 에스프레소 등 다양한 커피 원두에 혼합해 사용하는 제품인 만큼 국내 자영업자들에 타격이 큰 편이다.
광주·전남 지역 카페에 원두를 제공하는 도매업체 관계자 최모(49)씨는 “국제 원두 가격이 오르면서 국내 원두값도 올랐다. 모든 품종 원두가 일제히 올랐지만 가장 심한 것은 로부스타 품종으로 지난 4월 초부터 500g 한 봉지 가격이 400원이나 뛰어올랐다”며 “소매가를 올리고 싶지만, 거래처 반발이 있으리라 예상해 올해를 지나고 내년부터 1000원 정도 올릴 예정이다”고 설명했다.
원두가격 상승은 개인카페를 운영하는 자영업자 외에도 프랜차이즈 카페 가맹점주들에게도 걱정거리다. 프랜차이즈 카페의 경우 대량으로 원두를 구매해 어느 정도 소비자가격 방어가 가능하지만, 판매가격 조정은 온전히 본사의 결정이라 원두 납품가는 올랐는데 가격은 그대로여서 가맹점주 사이 볼멘소리도 나오고 있다.
광주 북구와 남구에서 무인카페를 운영 중인 박모(32)씨는 “지난해 본사에서 원두값을 올렸지만 소비자 판매가는 조정해 주지 않아 상황이 어렵다”며 “올해 원두값이 치솟으면서 본사에서 또 납품가를 올릴 것으로 예상하지만 작년같이 판매가를 동결하게 되면 먹고 살길이 막막하다”고 토로했다.
이미 가격을 올린 커피 브랜드들도 있다.
커피 프랜차이즈인 하삼동커피, 더벤티, 커피빈 등이 원두, 설탕 등 원부자재 가격 상승과 임대료 및 인건비 영향으로 가격 인상을 감행했다.
하삼동커피는 지난 1일부터 카페라떼 등 음료 6종의 가격을 200원씩 올렸다. 더벤티도 지난달 22일부터 카페라떼 등 음료 7종의 가격을 200원~500원 올렸다. 커피빈은 올해 초 카페라떼 등 우유가 포함된 음료의 가격을 200원씩 인상했으며 지난달 17일부터는 편의점에서 판매되는 파우치 음료 가격도 100원씩 인상됐다.'''


In [ ]:
# 실습
articles=[article1, article2, article3]

for article in articles[:]:
    title_represent, total_coherence, matrix_df=coherence_within_docs(article, sent_max=False)
    #print(matrix_df)
    print(f'title_representativeness: {title_represent}, total_coherence:{total_coherence}')

## 문서(기사) 간 표절: 제목 간 비교와 전체 문장들 비교

In [ ]:
# 토픽모델링으로 클러스터링한 후 문서간 유사도 측정
# 위 함수들(문장 분리, 인코딩) 사용하며, 코사인 유사도 측정

from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
import pandas as pd
import numpy as np

# 모델 로드
s_model = SentenceTransformer('bongsoo/kpf-sbert-128d-v1')

def sim_between_docs(article1, article2, sent_max=10):  # 두 문서에서 앞에서(제목이 첫번째 문장) 몇개 문장까지 사용할 것인지 설정
    if sent_max:
        txts1, txts2=sent_split(article1)[:sent_max], sent_split(article2)[:sent_max]
    else:
        txts1, txts2=sent_split(article1), sent_split(article2)
    txts_1, txts_2=[txts1[0], ' '.join(txts1)], [txts2[0], ' '.join(txts2)]  # 각 문서별로 제목과 전체 2개를 포함
    encoded_txts = s_model.encode(txts_1+txts_2)
    sim_matrix=cosine_similarity_matrix(encoded_txts)     # 두 문서의 제목과 전체 간의 비교 추출
    matrix_df=round(pd.DataFrame(sim_matrix, columns=['제목1','전체1','제목2','전체2']), 4)
    matrix_df.index=['제목1','전체1','제목2','전체2']
    title_sim=round(matrix_df.iloc[0,2], 4)   # 기사들의 첫번째 문장이 title이라고 할 때, title 간의 유사도 > 제목 간 유사도
    total_sim=round(matrix_df.iloc[1,3], 4)    # 제목을 포함해 모든 문장들 간의 유사도 평균 -> 기사 간 유사도
    return (title_sim, total_sim, matrix_df)


In [ ]:
# 실습

articles=[article1, article2, article3]

for i in range(len(articles)):                  # pair로 만들어 유사도 산출
    for j in range(i + 1, len(articles)):
        title_sim, total_sim, matrix_df=sim_between_docs(articles[i],articles[j], sent_max=10)
        #print(matrix_df)
        print(f'{str(i+1)}-{str(j+1)}: title_sim={title_sim}, total_sim={total_sim}')

# KPF-BERT 활용: 위 sbert 사용 권장. 이 내용은 참조만.
- SBERT는 BERT의 변형으로, 문장 레벨의 임베딩을 효과적으로 생성하기 위해 설계되었음.
- BERT 모델은 두 문장 간의 유사도를 계산할 때 상대적으로 비효율적이며, 계산 비용이 많이 듦.
- 작동 방식: SBERT는 BERT를 기반으로 하지만 Siamese network와 Triplet network 아키텍처를 사용하여 문장 임베딩을 최적화. 이러한 네트워크는 유사한 문장은 가깝게, 다른 문장은 멀리 떨어지게 임베딩 공간에 배치

## 문서(기사) 내 일관성2: 제목의 대표성과 문장들의 수렴성(KPF-BERT 사용)

In [ ]:
import pandas as pd
import numpy as np
from scipy.spatial.distance import cosine

def coherence_within_doc2(article, sent_max=10):   # 문서에서 앞에서(제목이 첫번째 문장) 몇개 문장까지 사용할 것인지 설정
    if sent_max:
        txts=sent_split(article)[:sent_max]
    else:
        txts=sent_split(article)
    encoded_txts=[encode(t) for t in txts]
    sim_matrix=cosine_similarity_matrix(encoded_txts)
    sim_matrix[sim_matrix == 1] = np.nan
    matrix_df=round(pd.DataFrame(sim_matrix), 4)
    title_represent=round(matrix_df.iloc[0].mean(), 4)   # 기사의 첫번째 문장이 title이라고 할 때, title이 다른 문장들과 맺는 유사도 평균 -> 제목의 내용 요약 수준(대표성)
    total_coherence=round(matrix_df.mean().mean(), 4)    # 제목을 포함해 모든 문장들 간의 유사도 평균 -> 기사의 의미상 일관성(수렴성) 수준
    return (title_represent, total_coherence, matrix_df)

# 실습
title_represent, total_coherence, matrix_df=coherence_within_doc2(article1, sent_max=False)
print(matrix_df)
print(f'title_representativeness: {title_represent}, total_coherence:{total_coherence}')

## 문서(기사) 간 표절2: 제목 간 비교와 전체 문장들 비교(KPF-BERT 사용)

In [ ]:
# 토픽모델링으로 클러스터링한 후 문서간 유사도 측정
# 위 함수들(문장 분리, 인코딩) 사용하며, 코사인 유사도 측정

import pandas as pd
import numpy as np
from scipy.spatial.distance import cosine

def sim_between_docs2(article1, article2, sent_max=10):  # 두 문서에서 앞에서(제목이 첫번째 문장) 몇개 문장까지 사용할 것인지 설정
    if sent_max:
        txts1, txts2=sent_split(article1)[:sent_max], sent_split(article2)[:sent_max]
    else:
        txts1, txts2=sent_split(article1), sent_split(article2)
    encoded_txts1, encoded_txts2=[encode(t) for t in txts1], [encode(t) for t in txts2]
    sim_matrix=cosine_similarity_matrix(encoded_txts1+encoded_txts2)[:len(encoded_txts1), -len(encoded_txts2):]
    matrix_df=round(pd.DataFrame(sim_matrix), 4)
    title_sim=round(matrix_df.iloc[0].mean(), 4)   # 기사들의 첫번째 문장이 title이라고 할 때, title 간의 유사도 > 제목 간 유사도
    total_sim=round(matrix_df.mean().mean(), 4)    # 제목을 포함해 모든 문장들 간의 유사도 평균 -> 기사 간 유사도
    return (title_sim, total_sim, matrix_df)


# 실습
title_sim, total_sim, matrix_df=sim_between_docs2(article1, article2, sent_max=10)
print(matrix_df)
print(f'title_sim: {title_sim}, total_sim:{total_sim}')